# Coding with Claude

### Import Libraries

In [1]:
import os
import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

if ANTHROPIC_API_KEY is None:
  raise Exception('Missing anthropic api key')

### Call the api

##### What is temperature
- Low (0.1 - 0.4) - More detrministic
- Medium (0.5 - 0.7) - Balanced
- High (0.8 - 1.0) - Very creative, verbose (can be off tone)

In [4]:
client = anthropic.Anthropic()

message = client.messages.create(
  model = 'claude-3-7-sonnet-20250219',
  max_tokens = 1000,
  temperature = 1,
  system = 'You are a world-class poet. Respond only with short poems.',
  messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Why is the ocean salty?"
            }
        ]
    }
  ]
)

responseContent = message.content

In [6]:
print(responseContent[0].text)

# Salt of the Deep

Tears of the earth,
Minerals flowing free,
Rivers carry secrets
To the ancient sea.

Eons of erosion,
Time's patient alchemy,
The ocean keeps collecting
What land sets free.


### Customize the prompt

##### Customer service representative

In [12]:
complaint = "You have overcharged me for this product. I demand a refund"

In [13]:
system_message = "You are an expert customer service representative, Respond thoughfully and informatively."

prompt = f""" 
Write a brief professional email response to an unhappy customer with the following issue:

{complaint}

Requirements:
- Keep under 150 words
- Maintain a professional but empathetic tone
- Acknowledge the issue clearly
- Offer a specific solution
- Include a follow-up action
"""

In [14]:
message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=0.7,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]
)

complaintEMailResponseContent = message.content[0]

In [15]:
display(Markdown(complaintEMailResponseContent.text))

# Subject: Regarding Your Refund Request - We're Here to Help

Dear Valued Customer,

Thank you for bringing this overcharging issue to our attention. I sincerely apologize for the frustration this has caused you.

I've reviewed your account and understand your concern completely. We take billing accuracy very seriously, and I'm committed to resolving this promptly for you.

I've initiated a full refund for the overcharged amount, which should appear in your account within 3-5 business days. Additionally, I'd like to offer you a 10% discount on your next purchase as a gesture of goodwill.

To ensure this doesn't happen again, our billing team will conduct a thorough review of your account.

Could you please confirm receipt of the refund by the end of next week? If you don't see it by then, please contact me directly at support@company.com.

Sincerely,
Customer Service Representative

### Rate the response using a response model

In [16]:
system_message = "You are an expert customer service representative. Evaluate the email response to be sent to a customer complaint."

reasoning_prompt = f"""
A customer has mad a complaint

{complaint}

Evaluate this customer service email response

{complaintEMailResponseContent.text}

Rate each category with ONE LINE explanation

1. Professionalism (1-10): [Score] - [One-line explanation]
2. Empathy (1-10): [Score] - [One-line explanation]
3. Problem Resolution (1-10): [Score] - [One-line explanation]
4. Clarity (1-10): [Score] - [One-line explanation]
5. Brand Representation (1-10): [Score] - [One-line explanation]

TOTAL SCORE: [X/100]
"""

In [17]:
message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=5000,
    thinking={
        "type": "enabled",
        "budget_tokens": 3000
    },
    temperature=1,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": reasoning_prompt
                }
            ]
        }
    ]
)
reasoning_response = message.content
print(reasoning_response)

[ThinkingBlock(signature='ErUBCkYIBRgCIkA4DNcpn8M9/PV/aDLPgB1RZ5skhW/718UjRPV3ilkEhxAXKccF5l7WSWk8e1u466OoD+To4e2gqqQI5Y2bkAOeEgwUv1SH7V9I8+vjVaQaDNzPde1IygywKkuAlSIwo9k1qsVhx2LXZL9+u0r7LGfno8qie0XvLkE3/dArheExaWLjOppQOmgKih54vdG+Kh1eoCSC9lnerSYLpVouxQ/Hn0ZSjuucjKlDmmiaTRgC', thinking='Let me evaluate this customer service email response based on the five criteria requested:\n\n1. Professionalism (1-10):\n- The email uses a professional tone and language\n- Has a clear subject line and formal greeting/closing\n- Uses proper grammar and punctuation\n- Avoids defensive language\n- Takes responsibility for the issue\n- Includes contact information for follow-up\nI would rate this a 9/10. The response is highly professional, well-structured, and maintains appropriate business language throughout.\n\n2. Empathy (1-10):\n- Begins with an apology acknowledging the customer\'s frustration\n- Shows understanding of the customer\'s concern ("I understand your concern completely")\n- Takes the co

In [18]:
display(Markdown(reasoning_response[0].thinking))

Let me evaluate this customer service email response based on the five criteria requested:

1. Professionalism (1-10):
- The email uses a professional tone and language
- Has a clear subject line and formal greeting/closing
- Uses proper grammar and punctuation
- Avoids defensive language
- Takes responsibility for the issue
- Includes contact information for follow-up
I would rate this a 9/10. The response is highly professional, well-structured, and maintains appropriate business language throughout.

2. Empathy (1-10):
- Begins with an apology acknowledging the customer's frustration
- Shows understanding of the customer's concern ("I understand your concern completely")
- Takes the complaint seriously ("We take billing accuracy very seriously")
- Offers a goodwill gesture (10% discount) beyond just fixing the problem
I would rate this an 8/10. The response shows good empathy, but could perhaps acknowledge the inconvenience caused a bit more specifically.

3. Problem Resolution (1-10):
- Clearly states that a refund has been initiated
- Provides a timeframe for the refund (3-5 business days)
- Offers additional compensation (10% discount)
- Mentions a plan to prevent recurrence ("our billing team will conduct a thorough review")
- Requests confirmation of resolution and provides a follow-up path
I would rate this a 9/10. The resolution is comprehensive and includes preventive measures.

4. Clarity (1-10):
- Clear explanation of what action has been taken
- Specific timeframe provided
- Next steps are outlined
- Contact information is provided
- Request for confirmation is clear
I would rate this a 9/10. The message is straightforward and easy to understand.

5. Brand Representation (1-10):
- Maintains a positive tone despite addressing a complaint
- Shows the company values customer feedback
- Demonstrates commitment to fixing issues promptly
- Shows generosity with the additional discount
- Emphasizes quality control and improvement
I would rate this an 8/10. The response represents the brand well as customer-focused and responsive.

Total score calculation:
If I simply add the scores: 9 + 8 + 9 + 9 + 8 = 43/50
If we're converting to a score out of 100: (43/50) × 100 = 86/100

However, the prompt asks for a total out of 100, so I should consider each category worth 20 points maximum:
(9/10×20) + (8/10×20) + (9/10×20) + (9/10×20) + (8/10×20) = 18 + 16 + 18 + 18 + 16 = 86/100

In [20]:
display(Markdown(reasoning_response[1].text))

# Evaluation of Customer Service Email Response

1. Professionalism (1-10): 9 - Maintains appropriate business tone with proper structure, grammar, and a clear subject line that addresses the issue directly.

2. Empathy (1-10): 8 - Opens with a sincere apology and acknowledges customer frustration, though could be slightly more specific about understanding the impact of overcharging.

3. Problem Resolution (1-10): 9 - Provides clear action (refund initiated), timeline, additional compensation, preventive measures, and follow-up instructions.

4. Clarity (1-10): 9 - Communicates clearly what has been done, what will happen next, and includes specific timeframes and contact information.

5. Brand Representation (1-10): 8 - Demonstrates company values through prompt resolution, goodwill gesture, and commitment to improvement.

TOTAL SCORE: 86/100

This is an excellent response that addresses the customer's complaint directly while maintaining professionalism. The refund process is clearly explained with specific timelines, and the additional discount shows the company values the customer relationship beyond just fixing the immediate problem. The response also builds in accountability by requesting confirmation and offering direct contact information.

### Improve rating - challenge (use empathy)

- LLM's lack emotions
- An empathy phrase front loads the emotional context
- Improves attentivness and warmth in response
- Improves scores for Empathy & Brand representation categories

In [25]:
empathy_prompt = """
Generate 3 short, professional yet warm sentences that:
- Show empathy for an overcharge issue
- Reinforce brand values like trust and helpful service
- Can be inserted into a customer service email

Each should be under 25 words.
"""

In [29]:
empathy_result = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    temperature=0.7,
    max_tokens=200,
    system="You are a helpful assistant generating goodwill phrases.",
    messages=[
        {"role": "user", "content": empathy_prompt}
    ]
)
empathy_content = empathy_result.content

In [ ]:
def best_empathy(empathy_lines, client):
    prompt = f"""
    You are an expert in customer communication. These are three empathy statements generated for a customer support email.

    Rate and rank each line for tone, relevance, and sincerity. Choose the BEST one that is professional, emotionally intelligent, and applicable to most situations.

    Empathy Lines:
    1. {empathy_lines[0]}
    2. {empathy_lines[1]}
    3. {empathy_lines[2]}

    Respond in this format:

    BEST: [1, 2, or 3] - [Copy of the selected line]
    WHY: [One-line explanation why this line is the best]
    """

    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=300,
        temperature=0, # asking model to be deterministic and consistent, useful when selection, scoring, or comparison tasks,
        system="You are a professional assistant helping select the best empathy tone for customer emails.",
        messages=[{
            "role": "user",
            "content": [{"type": "text", "text": prompt}]
        }]
    )

    reply = response.content[0].text.strip()
    print(response.content[0].text.strip())
    if "BEST:" in reply:
        best_line = reply.split("BEST:")[1].split("WHY:")[0].strip().split(" - ")[-1].strip()
        return best_line
    else:
        return empathy_lines[0]  # fallback


In [37]:
print(empathy_result.content[0])
print(empathy_result.content[0].text)
empathy_sentences = empathy_result.content[0].text.strip().split("\n")
chosen_empathy = best_empathy(empathy_sentences, client)


TextBlock(citations=None, text='1. We sincerely apologize for the overcharge and understand how frustrating unexpected charges can be on your account.\n\n2. At [Company], we value your trust above all else and are committed to making this right promptly.\n\n3. Our team is here to help resolve this issue quickly and ensure your experience reflects the reliable service you deserve.', type='text')
1. We sincerely apologize for the overcharge and understand how frustrating unexpected charges can be on your account.

2. At [Company], we value your trust above all else and are committed to making this right promptly.

3. Our team is here to help resolve this issue quickly and ensure your experience reflects the reliable service you deserve.
I notice there are only two empathy statements provided (numbered 1 and 2), rather than three as mentioned in the instructions.

BEST: 1 - We sincerely apologize for the overcharge and understand how frustrating unexpected charges can be on your account.


In [38]:
system_message = "You are an expert customer service representative. Respond thoughtfully, empathetically, and clearly."

prompt = f""" 
Write a professional and empathetic email response to a customer with the following complaint:

{complaint}

Requirements:
- Keep under 150 words
- Maintain a professional but empathetic tone
- Acknowledge the issue clearly
- Offer a specific solution
- Include a follow-up action
- Reinforce trust and brand commitment

Incorporate the following empathy phrase near the beginning:

"{chosen_empathy}"
"""

In [39]:
message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=0.7,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]
)

complaintEmailWithChosenEmpatyResponseContent = message.content[0]

In [40]:
display(Markdown(complaintEmailWithChosenEmpatyResponseContent.text))

# Subject: Regarding Your Recent Overcharge Concern

Dear Valued Customer,

We sincerely apologize for the overcharge and understand how frustrating unexpected charges can be on your account. Thank you for bringing this matter to our attention.

We've reviewed your account and confirmed the billing discrepancy. We will process a full refund for the overcharged amount, which should appear in your account within 3-5 business days.

To prevent similar issues in the future, we're implementing additional verification steps in our billing process. We've also added a 10% discount to your next purchase as a gesture of our appreciation for your patience.

Our customer service manager will follow up with you directly by tomorrow to confirm the refund has been processed.

Your trust is important to us, and we're committed to making this right.

Sincerely,

Customer Service Team

In [41]:
system_message = "You are an expert customer service representative. Evaluate the email response to be sent to a customer complaint."

scoring_prompt = f"""
A customer made the following complaint:

{complaint}

Evaluate the customer service email below:

{complaintEmailWithChosenEmpatyResponseContent.text}

Use the following rubric to score each category (1-10) with a one-line explanation:

1. **Professionalism** – Is the tone appropriate and respectful?
2. **Empathy** – Does the response show emotional awareness and understanding?
3. **Problem Resolution** – Is there a specific, actionable solution?
4. **Clarity** – Is the message easy to follow and well-structured?
5. **Brand Representation** – Does the message reflect a helpful, trustworthy brand?

Each explanation should be one concise sentence.

TOTAL SCORE: [X/100]
"""

In [42]:
message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=5000,
    thinking={
        "type": "enabled",
        "budget_tokens": 3000
    },
    temperature=1,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": scoring_prompt
                }
            ]
        }
    ]
)
reasoning_response = message.content
print(reasoning_response)

[ThinkingBlock(signature='ErUBCkYIBRgCIkAaVVFjJOu/BBoOaLYs0AY5C1YuWhtePbZ+8sGzLfLy+xQhOEkwQF1fE9R/mhySdSVhLdymDf9EM+QlhDJ7kuLWEgwasM8yagaGVEvHx3AaDBH9Aj8BYhfgd68IIyIw7P2lkWNZmAOfOIwPyJwNVXRMhNlwTb6dxoM6UCPw3WQSt1jtGwndLuODexS/N+JFKh3zWjRmum/X7Jg2fPNlC3JsDYlG0cvlj3Jnp01YsRgC', thinking="Let me evaluate this customer service email response based on the provided rubric. I'll assess each category on a scale of 1-10 and provide a one-line explanation.\n\n1. **Professionalism** (1-10)\nThe email uses a professional tone throughout, with appropriate salutation and closing. It avoids defensive language, maintains formality without being cold, and uses proper grammar and structure. The subject line is clear and professional. The tone is respectful and acknowledges the customer's concern without questioning their claim.\nScore: 9/10\n\n2. **Empathy** (1-10)\nThe response begins with an apology and acknowledges the customer's frustration with unexpected charges. It shows understanding of the inco

In [43]:
display(Markdown(reasoning_response[0].thinking))

Let me evaluate this customer service email response based on the provided rubric. I'll assess each category on a scale of 1-10 and provide a one-line explanation.

1. **Professionalism** (1-10)
The email uses a professional tone throughout, with appropriate salutation and closing. It avoids defensive language, maintains formality without being cold, and uses proper grammar and structure. The subject line is clear and professional. The tone is respectful and acknowledges the customer's concern without questioning their claim.
Score: 9/10

2. **Empathy** (1-10)
The response begins with an apology and acknowledges the customer's frustration with unexpected charges. It shows understanding of the inconvenience caused and validates the customer's feelings. The email goes beyond just acknowledging the issue by offering a gesture (10% discount) and promising follow-up, showing they value the customer's experience.
Score: 9/10

3. **Problem Resolution** (1-10)
The email clearly states they've confirmed the billing discrepancy and provides a specific solution (full refund) with a timeline (3-5 business days). It also mentions preventative measures for the future and provides a follow-up mechanism (manager will contact by tomorrow). The resolution is complete and addresses immediate concerns as well as future prevention.
Score: 10/10

4. **Clarity** (1-10)
The message is well-structured with clear paragraphs, each serving a purpose. It's concise yet complete, avoids jargon, and clearly communicates what happened and what will happen next. The timeline and expectations are explicitly stated.
Score: 10/10

5. **Brand Representation** (1-10)
The response portrays the company as responsible (by admitting the mistake), proactive (by implementing new verification steps), generous (by offering a discount), and customer-focused (by promising personal follow-up). It concludes with a commitment statement that reinforces brand values of trust and accountability.
Score: 10/10

To calculate the total score, I'll multiply each category score by 2 (since there are 5 categories and the total is out of 100):
- Professionalism: 9 × 2 = 18
- Empathy: 9 × 2 = 18
- Problem Resolution: 10 × 2 = 20
- Clarity: 10 × 2 = 20
- Brand Representation: 10 × 2 = 20

Total: 18 + 18 + 20 + 20 + 20 = 96/100

In [44]:
display(Markdown(reasoning_response[1].text))

# Email Response Evaluation

1. **Professionalism**: 9/10
   The email maintains a formal yet approachable tone with proper structure and no defensive language.

2. **Empathy**: 9/10
   The response immediately acknowledges frustration and validates the customer's feelings about the unexpected charge.

3. **Problem Resolution**: 10/10
   Provides a specific solution (full refund) with clear timeline, preventative measures, and follow-up commitment.

4. **Clarity**: 10/10
   Information is logically organized with clear paragraphs, specific timelines, and straightforward language.

5. **Brand Representation**: 10/10
   The company appears accountable, proactive, and customer-focused through its admission, compensation, and commitment to improvement.

**TOTAL SCORE: 96/100**

This is an excellent response that addresses the customer's concern comprehensively while maintaining professionalism and showing genuine care for their experience.